In [1]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import tqdm
from scipy.stats import levene

# import anova and tukeyhsd
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [2]:
def anova_function(features_df: pd.DataFrame, Metdata_column: str) -> pd.DataFrame:
    """
    This function will take in a dataframe and a metadata column and return the results of an anova and tukeyhsd test for each feature.


    Parameters
    ----------
    features_df : pd.DataFrame
        The dataframe containing the features with only one metadata column
    Metdata_column : str
        The name of the metadata column to be used for the anova test

    Returns
    -------
    pd.DataFrame
        A dataframe containing the results of the anova and tukeyhsd test for each feature
    """

    # anova and tukeyhsd for each feature
    # create a pandas data frame to store the results
    anova_results = pd.DataFrame()

    # loop through each feature
    for feature in tqdm.tqdm(features_df.columns[:-1]):
        # create a model
        model = ols(f"{feature} ~ C({Metdata_column})", data=features_df).fit()
        # create an anova table
        anova_table = sm.stats.anova_lm(model, typ=2)
        # create a tukeyhsd table
        tukeyhsd = pairwise_tukeyhsd(features_df[feature], features_df[Metdata_column])
        # get the f-statistic based p-value
        anova_p_value = anova_table["PR(>F)"][0]
        tmp = pd.DataFrame(
            tukeyhsd._results_table.data, columns=tukeyhsd._results_table.data[0]
        ).drop(0)
        tmp.reset_index(inplace=True, drop=True)
        # drop the first row
        tmp["feature"] = feature
        tmp["anova_p_value"] = anova_p_value
        tmp = pd.DataFrame(tmp)

        anova_results = pd.concat([anova_results, tmp], axis=0).reset_index(drop=True)
    return anova_results

In [3]:
file_path = pathlib.Path("../../data/5.converted_data/sum_aggregated_data.parquet")
df = pd.read_parquet(file_path)
df.head()

,Metadata_genotype,Metadata_replicate,AreaShape_Area,AreaShape_CentralMoment_0_0,AreaShape_CentralMoment_0_1,AreaShape_CentralMoment_0_2,AreaShape_CentralMoment_0_3,AreaShape_CentralMoment_1_0,AreaShape_CentralMoment_1_1,AreaShape_CentralMoment_1_2,...,Texture_SumEntropy_OP_3_02_256,Texture_SumEntropy_OP_3_03_256,Texture_SumVariance_OP_3_00_256,Texture_SumVariance_OP_3_01_256,Texture_SumVariance_OP_3_02_256,Texture_SumVariance_OP_3_03_256,Texture_Variance_OP_3_00_256,Texture_Variance_OP_3_01_256,Texture_Variance_OP_3_02_256,Texture_Variance_OP_3_03_256
0,high,1,1.192966,1.192966,0.541753,-0.098475,0.781736,4.201830,-0.927874,-0.085964,...,-0.208210,-0.238073,-1.091712,-1.050054,-1.086609,-1.089178,-1.115127,-1.124203,-1.117953,-1.121744
1,high,10,2.477908,2.477908,1.319626,3.444728,-1.748595,0.358390,-3.289519,1.650540,...,1.293954,1.270712,1.261364,1.284182,1.187508,1.270548,1.115726,1.096947,1.121969,1.095539
2,high,11,4.262414,4.262414,1.261731,2.852939,0.583182,3.602414,-0.541325,-0.328613,...,1.373542,1.356461,1.104765,1.141646,1.052532,1.096949,0.997651,0.972537,1.000166,0.980426
3,high,12,2.402476,2.402476,0.049886,2.558238,-0.928171,-3.305706,-1.964619,0.215359,...,0.619009,0.617867,-0.109557,-0.119752,-0.196800,-0.135860,-0.207564,-0.218735,-0.202171,-0.213544
4,high,13,2.389777,2.389777,-1.108736,-0.068931,0.101765,-2.689960,-0.967345,-1.588868,...,0.113413,0.095483,-0.650341,-0.591348,-0.642077,-0.644871,-0.696817,-0.712509,-0.700572,-0.704335


In [4]:
# split the features and the metadata
metadata = df.columns.str.contains("Metadata")
# filter the metadata
metadata_df = df.loc[:, metadata]
# filter the features
features_df = df.loc[:, ~metadata]

## Anova for genotype only

In [5]:
anova_input_df = features_df.copy()
anova_input_df["Metadata_genotype"] = metadata_df["Metadata_genotype"]
anova_output_df = anova_function(anova_input_df, "Metadata_genotype")
print(anova_output_df.shape)
anova_output_df.head()

  0%|                                                                                                                 | 0/244 [00:00<?, ?it/s]

  0%|▍                                                                                                        | 1/244 [00:00<00:30,  8.00it/s]

  1%|▊                                                                                                        | 2/244 [00:00<00:29,  8.19it/s]

  1%|█▎                                                                                                       | 3/244 [00:00<00:28,  8.39it/s]

  2%|█▋                                                                                                       | 4/244 [00:00<00:28,  8.36it/s]

...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████▌| 243/244 [00:29<00:00,  8.33it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 244/244 [00:29<00:00,  8.34it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 244/244 [00:29<00:00,  8.21it/s]

(732, 9)


,group1,group2,meandiff,p-adj,lower,upper,reject,feature,anova_p_value
0,high,unsel,-3.4266,0.0,-4.6531,-2.2001,True,AreaShape_Area,4.479053e-12
1,high,wt,-5.1908,0.0,-6.4173,-3.9643,True,AreaShape_Area,4.479053e-12
2,unsel,wt,-1.7642,0.0033,-2.9907,-0.5377,True,AreaShape_Area,4.479053e-12
3,high,unsel,-3.4266,0.0,-4.6531,-2.2001,True,AreaShape_CentralMoment_0_0,4.479053e-12
4,high,wt,-5.1908,0.0,-6.4173,-3.9643,True,AreaShape_CentralMoment_0_0,4.479053e-12


In [6]:
# save the results
output_file = pathlib.Path(
    "../../data/6.analysis_results/sum_aggregated_anova_results.parquet"
)
output_file.parent.mkdir(exist_ok=True, parents=True)
anova_output_df.to_parquet(output_file)

## Levene's test for homogeneity of variance

In [7]:
# split the df into three genotypes
high_df = df[df["Metadata_genotype"] == "high"]
unsel_df = df[df["Metadata_genotype"] == "unsel"]
wt_df = df[df["Metadata_genotype"] == "wt"]
levene_test_results = {"feature": [], "levene_statistic": [], "levene_p_value": []}
for feature in tqdm.tqdm(features_df.columns):
    # calculate the levene test for each feature
    levene_results = levene(wt_df[feature], unsel_df[feature], high_df[feature])
    levene_test_results["feature"].append(feature)
    levene_test_results["levene_statistic"].append(levene_results.statistic)
    levene_test_results["levene_p_value"].append(levene_results.pvalue)

levene_test_results_df = pd.DataFrame(levene_test_results)
levene_test_results_df

  0%|                                                                                                                 | 0/244 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 244/244 [00:00<00:00, 4160.05it/s]

,feature,levene_statistic,levene_p_value
0,AreaShape_Area,2.514193,0.093966
1,AreaShape_CentralMoment_0_0,2.514193,0.093966
2,AreaShape_CentralMoment_0_1,1.881105,0.165992
3,AreaShape_CentralMoment_0_2,5.617614,0.007180
4,AreaShape_CentralMoment_0_3,1.489590,0.238011
...,...,...,...
239,Texture_SumVariance_OP_3_03_256,16.275324,0.000007
240,Texture_Variance_OP_3_00_256,15.940766,0.000009
241,Texture_Variance_OP_3_01_256,16.424118,0.000007
242,Texture_Variance_OP_3_02_256,15.899348,0.000009


## Calculate the levenes test statistic for the equality of variances

In [8]:
# split the df into three genotypes
high_df = df[df["Metadata_genotype"] == "high"]
unsel_df = df[df["Metadata_genotype"] == "unsel"]
wt_df = df[df["Metadata_genotype"] == "wt"]
group_dict = {
    "high_vs_unsel": [high_df, unsel_df],
    "high_vs_wt": [high_df, wt_df],
    "unsel_vs_wt": [wt_df, unsel_df],
    "all": [high_df, unsel_df, wt_df],
}


levene_test_results = {
    "feature": [],
    "levene_statistic": [],
    "levene_p_value": [],
    "group": [],
}
for group in tqdm.tqdm(group_dict.keys()):
    for feature in features_df.columns:
        # calculate the levene test for each feature
        if not group == "all":
            levene_results = levene(
                group_dict[group][0][feature], group_dict[group][1][feature]
            )
            levene_test_results["feature"].append(feature)
            levene_test_results["levene_statistic"].append(levene_results.statistic)
            levene_test_results["levene_p_value"].append(levene_results.pvalue)
            levene_test_results["group"].append(group)
        else:
            levene_results = levene(
                group_dict[group][0][feature],
                group_dict[group][1][feature],
                group_dict[group][2][feature],
            )
            levene_test_results["feature"].append(feature)
            levene_test_results["levene_statistic"].append(levene_results.statistic)
            levene_test_results["levene_p_value"].append(levene_results.pvalue)
            levene_test_results["group"].append(group)

levene_test_results_df = pd.DataFrame(levene_test_results)

# sort the levene test results levene_test_results_df
# change the levene p-value to a float
levene_test_results_df["levene_p_value"] = levene_test_results_df[
    "levene_p_value"
].astype(float)
levene_test_results_df = levene_test_results_df.sort_values(
    "levene_p_value", ascending=False
)
levene_test_results_df

  0%|                                                                                                                   | 0/4 [00:00<?, ?it/s]

/home/lippincm/miniconda3/envs/op_cell_processing_env/lib/python3.8/site-packages/scipy/stats/_morestats.py:2710: RuntimeWarning: invalid value encountered in scalar divide
  W = numer / denom


 75%|████████████████████████████████████████████████████████████████████████████████▎                          | 3/4 [00:00<00:00, 23.66it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 22.87it/s]

,feature,levene_statistic,levene_p_value,group
90,AreaShape_Zernike_8_4,0.000018,9.966091e-01,high_vs_unsel
540,AreaShape_NormalizedMoment_3_3,0.000046,9.946505e-01,unsel_vs_wt
920,RadialDistribution_ZernikePhase_OP_9_3,0.011160,9.889052e-01,all
248,AreaShape_CentralMoment_0_3,0.000264,9.871715e-01,high_vs_wt
188,RadialDistribution_ZernikePhase_OP_9_3,0.000338,9.854698e-01,high_vs_unsel
...,...,...,...,...
886,RadialDistribution_ZernikeMagnitude_OP_8_0,32.022343,5.912358e-09,all
268,AreaShape_HuMoment_4,75.849212,3.469854e-09,high_vs_wt
887,RadialDistribution_ZernikeMagnitude_OP_8_2,34.456337,2.403516e-09,all
900,RadialDistribution_ZernikePhase_OP_4_0,66.314773,2.826278e-13,all


In [9]:
# save the levene test results
# out dir
out_dir = pathlib.Path("../../data/6.analysis_results/")
# create the dir if it does not exist
out_dir.mkdir(parents=True, exist_ok=True)
levene_test_results_path = pathlib.Path(
    out_dir / "sum_aggregated_levene_test_results.csv"
)
levene_test_results_df.to_csv(levene_test_results_path)

### Calculate the levene test statistic for the aggregated data across feature types and genotypes

In [10]:
data_path = pathlib.Path(
    "../../data/5.converted_data/sum_aggregated_data.parquet"
).resolve(strict=True)
# Read the data
data = pd.read_parquet(data_path)

# Drop all metadata except for the genotype data
features_df = data.drop(columns=data.filter(like="Metadata").columns)
features_df["Metadata_genotype"] = data["Metadata_genotype"]


# turn the features into a long format
features_long_df = features_df.melt(
    id_vars="Metadata_genotype", var_name="feature", value_name="value"
)
features_long_df.head()
# Separate the feature into different parts
features_long_df[
    ["feature_group", "measurement", "bone", "parameter1", "parameter2", "parameter3"]
] = features_long_df["feature"].str.split("_", expand=True)

# Replace the Metadata_genotype with the actual genotype name
features_long_df["Metadata_genotype"] = features_long_df["Metadata_genotype"].replace(
    {"high": "High-Severity", "unsel": "Mid-Severity", "wt": "Wild Type"}
)
features_long_df.head()

,Metadata_genotype,feature,value,feature_group,measurement,bone,parameter1,parameter2,parameter3
0,High-Severity,AreaShape_Area,1.192966,AreaShape,Area,None,None,None,None
1,High-Severity,AreaShape_Area,2.477908,AreaShape,Area,None,None,None,None
2,High-Severity,AreaShape_Area,4.262414,AreaShape,Area,None,None,None,None
3,High-Severity,AreaShape_Area,2.402476,AreaShape,Area,None,None,None,None
4,High-Severity,AreaShape_Area,2.389777,AreaShape,Area,None,None,None,None


In [11]:
# break each genotype and featuretype into a separate dataframe
high_df = features_long_df[features_long_df["Metadata_genotype"] == "High-Severity"]
unsel_df = features_long_df[features_long_df["Metadata_genotype"] == "Mid-Severity"]
wt_df = features_long_df[features_long_df["Metadata_genotype"] == "Wild Type"]

# each feature group
high_df_AreaShape = high_df[high_df["feature_group"] == "AreaShape"]
high_df_Intensity = high_df[high_df["feature_group"] == "Intensity"]
high_df_Neighbors = high_df[high_df["feature_group"] == "Neighbors"]
high_df_radial = high_df[high_df["feature_group"] == "RadialDistribution"]
high_df_Granularity = high_df[high_df["feature_group"] == "Granularity"]

unsel_df_AreaShape = unsel_df[unsel_df["feature_group"] == "AreaShape"]
unsel_df_Intensity = unsel_df[unsel_df["feature_group"] == "Intensity"]
unsel_df_Neighbors = unsel_df[unsel_df["feature_group"] == "Neighbors"]
unsel_df_radial = unsel_df[unsel_df["feature_group"] == "RadialDistribution"]
unsel_df_Granularity = unsel_df[unsel_df["feature_group"] == "Granularity"]

wt_df_AreaShape = wt_df[wt_df["feature_group"] == "AreaShape"]
wt_df_Intensity = wt_df[wt_df["feature_group"] == "Intensity"]
wt_df_Neighbors = wt_df[wt_df["feature_group"] == "Neighbors"]
wt_df_radial = wt_df[wt_df["feature_group"] == "RadialDistribution"]
wt_df_Granularity = wt_df[wt_df["feature_group"] == "Granularity"]

# levene test for each feature group
levene_test_results = {
    "feature_group": [],
    "levene_statistic": [],
    "levene_p_value": [],
    "group": [],
}

group_dict = {
    "AreaShape": {
        "high_area_v_unsel_area": [high_df_AreaShape, unsel_df_AreaShape],
        "high_area_v_wt_area": [high_df_AreaShape, wt_df_AreaShape],
        "unsel_area_v_wt_area": [wt_df_AreaShape, unsel_df_AreaShape],
    },
    "Intensity": {
        "high_intensity_v_unsel_intensity": [high_df_Intensity, unsel_df_Intensity],
        "high_intensity_v_wt_intensity": [high_df_Intensity, wt_df_Intensity],
        "unsel_intensity_v_wt_intensity": [wt_df_Intensity, unsel_df_Intensity],
    },
    "Neighbors": {
        "high_neighbors_v_unsel_neighbors": [high_df_Neighbors, unsel_df_Neighbors],
        "high_neighbors_v_wt_neighbors": [high_df_Neighbors, wt_df_Neighbors],
        "unsel_neighbors_v_wt_neighbors": [wt_df_Neighbors, unsel_df_Neighbors],
    },
    "RadialDistribution": {
        "high_radial_v_unsel_radial": [high_df_radial, unsel_df_radial],
        "high_radial_v_wt_radial": [high_df_radial, wt_df_radial],
        "unsel_radial_v_wt_radial": [wt_df_radial, unsel_df_radial],
    },
    "Granularity": {
        "high_granularity_v_unsel_granularity": [
            high_df_Granularity,
            unsel_df_Granularity,
        ],
        "high_granularity_v_wt_granularity": [high_df_Granularity, wt_df_Granularity],
        "unsel_granularity_v_wt_granularity": [wt_df_Granularity, unsel_df_Granularity],
    },
}

for feature_group in tqdm.tqdm(group_dict.keys()):
    for group in group_dict[feature_group].keys():
        if not group == "all":
            levene_results = levene(
                group_dict[feature_group][group][0]["value"],
                group_dict[feature_group][group][1]["value"],
            )
            # calculate the variance for each feature group

            levene_test_results["feature_group"].append(feature_group)
            levene_test_results["levene_statistic"].append(levene_results.statistic)
            levene_test_results["levene_p_value"].append(levene_results.pvalue)
            levene_test_results["group"].append(group)
        else:
            pass

levene_test_results_df = pd.DataFrame(levene_test_results)
levene_test_results_df.head()

  0%|                                                                                                                   | 0/5 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 1432.48it/s]

,feature_group,levene_statistic,levene_p_value,group
0,AreaShape,0.000308,9.860078e-01,high_area_v_unsel_area
1,AreaShape,2.376294,1.233049e-01,high_area_v_wt_area
2,AreaShape,2.239614,1.346296e-01,unsel_area_v_wt_area
3,Intensity,2.984981,8.477890e-02,high_intensity_v_unsel_intensity
4,Intensity,82.922691,3.550921e-18,high_intensity_v_wt_intensity


In [12]:
# save the levene test results
# out dir
out_dir = pathlib.Path("../../data/6.analysis_results/")
# create the dir if it does not exist
out_dir.mkdir(parents=True, exist_ok=True)
levene_test_results_path = pathlib.Path(
    out_dir / "sum_aggregated_levene_test_results_feature_types.csv"
)
levene_test_results_df.to_csv(levene_test_results_path, index=False)

In [13]:
# Drop all metadata except for the genotype data
features_df = data.drop(columns=data.filter(like="Metadata").columns)
features_df["Metadata_genotype"] = data["Metadata_genotype"]

# turn the features into a long format
features_long_df = features_df.melt(
    id_vars="Metadata_genotype", var_name="feature", value_name="value"
)
features_long_df

,Metadata_genotype,feature,value
0,high,AreaShape_Area,1.192966
1,high,AreaShape_Area,2.477908
2,high,AreaShape_Area,4.262414
3,high,AreaShape_Area,2.402476
4,high,AreaShape_Area,2.389777
...,...,...,...
10243,wt,Texture_Variance_OP_3_03_256,5.298012
10244,wt,Texture_Variance_OP_3_03_256,4.250143
10245,wt,Texture_Variance_OP_3_03_256,4.989335
10246,wt,Texture_Variance_OP_3_03_256,6.360684


In [14]:
# get the variance for each feature group
var_df = features_long_df.groupby(["Metadata_genotype", "feature"]).var().reset_index()
var_df.head()
# change the value column name to variance
var_df.rename(columns={"value": "variance"}, inplace=True)

In [15]:
var_df[
    ["feature_group", "measurement", "bone", "parameter1", "parameter2", "parameter3"]
] = var_df["feature"].str.split("_", expand=True)

# Replace the Metadata_genotype with the actual genotype name
var_df["Metadata_genotype"] = var_df["Metadata_genotype"].replace(
    {"high": "High-Severity", "unsel": "Mid-Severity", "wt": "Wild Type"}
)
var_df
var_df = var_df.drop(
    columns=["feature", "measurement", "bone", "parameter1", "parameter2", "parameter3"]
)
var_df
# save the variance results
var_path = pathlib.Path(out_dir / "sum_aggregated_variance_results_feature_types.csv")
var_df.to_csv(var_path, index=False)

In [16]:
# get the mean and stdev for each feature group's variance
var_df = (
    var_df.groupby(["Metadata_genotype", "feature_group"])
    .agg(["mean", "std", "max", "min", "count"])
    .reset_index()
)
# ungroup the columns
var_df.columns = ["_".join(col).strip() for col in var_df.columns.values]
# rename the Metadata_genotype_ column and the feature_group_ column
var_df.rename(
    columns={
        "Metadata_genotype_": "Metadata_genotype",
        "feature_group_": "feature_group",
    },
    inplace=True,
)
var_df

,Metadata_genotype,feature_group,variance_mean,variance_std,variance_max,variance_min,variance_count
0,High-Severity,AreaShape,1.696818,1.402680,8.228906,0.000009,98
1,High-Severity,Granularity,0.472217,0.548388,1.539938,0.043669,6
2,High-Severity,Intensity,1.322782,0.771950,3.331811,0.196161,15
3,High-Severity,Neighbors,1.908916,0.733588,2.668279,1.204154,3
4,High-Severity,RadialDistribution,1.527171,1.035597,5.207460,0.000000,70
5,High-Severity,Texture,0.792697,0.508977,1.702177,0.058432,52
6,Mid-Severity,AreaShape,3.624904,1.608754,8.625774,0.848783,98
7,Mid-Severity,Granularity,2.468493,1.770383,5.747638,0.970630,6
8,Mid-Severity,Intensity,3.273020,1.800369,7.563879,1.235410,15
9,Mid-Severity,Neighbors,7.405256,3.995037,11.900394,4.260122,3


In [17]:
# save the variance results
var_path = pathlib.Path(
    out_dir / "sum_aggregated_variance_results_feature_types_stats.csv"
)
var_df.to_csv(var_path, index=False)